In [4]:
import json
import math
from datetime import datetime
from typing import Dict, List, Any, Tuple, Optional
import os
import logging
import boto3

logger = logging.getLogger()
logger.setLevel(logging.INFO)

S3_BUCKET = os.environ.get("collegefootballranks", "collegefootballranks")
TEAMS_KEY = "static-assets/teams.json"
s3_client = boto3.client("s3")


# --------------------------------------------------------------------
# Configuration: tweak these knobs to control behavior
# --------------------------------------------------------------------

CONFIG = {
    # base model
    "home_field_adv_init": 3.0,          # starting guess for HFA (points)
    "margin_scale": 0.25,                # strength of margin-of-victory effect
    "recency_half_life_days": 21.0,      # how fast games "decay" in importance
    "learning_rate": 0.01,
    "num_epochs": 400,
    "max_rating_abs": 5.0,               # clamp rating to [-5, 5] to avoid blowups

    # win–loss emphasis
    "wl_weight": 2.0,                    # how strongly win_pct nudges the rating
    "undefeated_min_games": 8,           # minimum games for undefeated bonus
    "undefeated_bonus": 0.75,            # flat bump for undefeated teams

    # head-to-head swap rules
    "h2h_swap_margin": 0.50,             # only swap if rating difference < this
}


# --------------------------------------------------------------------
# Utility helpers
# --------------------------------------------------------------------

def load_from_s3(KEY):
    """
    Load teams from S3 /static-assets/teams.json.
    Falls back to SAMPLE_TEAMS on any error.
    """
    try:
        logger.info(f"Loading teams from s3://{S3_BUCKET}/{KEY}")
        obj = s3_client.get_object(Bucket=S3_BUCKET, Key=KEY)
        body = obj["Body"].read()
        body = json.loads(body)

        if not isinstance(body, list):
            raise ValueError("teams.json is not a JSON list")

        logger.info(f"Loaded {len(body)} {KEY} from S3")
        return body

    except Exception as e:
        logger.exception("Failed to load teams from S3; using SAMPLE_TEAMS fallback")
        return SAMPLE_TEAMS


def parse_date(d: str) -> datetime:
    """Parse YYYY-MM-DD (or ISO like YYYY-MM-DDTHH:MM:SSZ) into datetime."""
    if len(d) >= 10 and d[4] == "-" and d[7] == "-":
        # handle "YYYY-MM-DD" or "YYYY-MM-DDTHH:MM:SSZ"
        return datetime.fromisoformat(d[:10])
    else:
        # fallback, may raise
        return datetime.fromisoformat(d)


def logistic(x: float) -> float:
    if x > 50:
        return 1.0
    if x < -50:
        return 0.0
    return 1.0 / (1.0 + math.exp(-x))


# --------------------------------------------------------------------
# Core model: fit base ratings from game results
# --------------------------------------------------------------------

def build_team_index(teams: List[Dict[str, Any]]) -> Dict[int, int]:
    """Map team_id -> index in rating vector."""
    # ensure deterministic ordering by sorting team_id
    sorted_ids = sorted(t["team_id"] for t in teams)
    return {tid: i for i, tid in enumerate(sorted_ids)}


def compute_latest_game_date(games: List[Dict[str, Any]]) -> Optional[datetime]:
    if not games:
        return None
    return max(parse_date(g["date"]) for g in games)


def game_weight(game: Dict[str, Any],
                latest_date: datetime,
                cfg: Dict[str, Any]) -> float:
    """
    Combine recency and margin into a single weight term.

    - Recency: exponential decay with half-life in days.
    - Margin: log(1 + margin) scaled.
    """
    game_date = parse_date(game["date"])
    age_days = max(0.0, (latest_date - game_date).days)
    half_life = cfg["recency_half_life_days"]
    if half_life <= 0:
        time_factor = 1.0
    else:
        time_factor = 0.5 ** (age_days / half_life)

    home_score = game["home_score"]
    away_score = game["away_score"]
    margin = abs(home_score - away_score)

    # Margin factor: 1 for 0–1 point, growing but with diminishing returns
    margin_factor = math.log(1.0 + margin) / math.log(1.0 + 21.0)  # 3-TD win ~ 1.0
    margin_factor = max(0.25, margin_factor)  # ensure close games still matter

    return time_factor * margin_factor


def margin_effect(game: Dict[str, Any], cfg: Dict[str, Any]) -> float:
    """
    Signed margin effect that nudges the logit in the direction of the winner.
    """
    home_score = game["home_score"]
    away_score = game["away_score"]
    margin = home_score - away_score
    if margin == 0:
        return 0.0
    s = math.copysign(1.0, margin)
    mag = math.log(1.0 + abs(margin))
    return cfg["margin_scale"] * s * mag


def fit_base_ratings(
    teams: List[Dict[str, Any]],
    games: List[Dict[str, Any]],
    cfg: Dict[str, Any]
) -> Tuple[Dict[int, float], float]:
    """
    Fit per-team base ratings + a single home-field-advantage term using
    simple gradient descent on a Bradley–Terry–style log-likelihood.
    """
    if not games:
        # no games => everyone at 0, HFA=0
        return {t["team_id"]: 0.0 for t in teams}, 0.0

    team_index = build_team_index(teams)
    n = len(team_index)
    ratings = [0.0] * n
    hfa = cfg["home_field_adv_init"]

    latest_date = compute_latest_game_date(games)
    game_weights = [game_weight(g, latest_date, cfg) for g in games]
    margin_effects = [margin_effect(g, cfg) for g in games]

    lr = cfg["learning_rate"]
    num_epochs = cfg["num_epochs"]
    max_abs = cfg["max_rating_abs"]

    for epoch in range(num_epochs):
        grad_r = [0.0] * n
        grad_hfa = 0.0

        for g, w, m_eff in zip(games, game_weights, margin_effects):
            h_id = g["home_team_id"]
            a_id = g["away_team_id"]
            h_idx = team_index[h_id]
            a_idx = team_index[a_id]

            r_h = ratings[h_idx]
            r_a = ratings[a_idx]

            is_neutral = bool(g.get("neutral_site", 0))
            hfa_term = 0.0 if is_neutral else hfa

            # logit = r_home + HFA - r_away + margin_effect
            logit = (r_h + hfa_term - r_a) + m_eff
            p_home = logistic(logit)

            home_score = g["home_score"]
            away_score = g["away_score"]
            if home_score == away_score:
                # treat ties as 0.5
                y = 0.5
            else:
                y = 1.0 if home_score > away_score else 0.0

            # gradient of log-likelihood wrt r_h, r_a, hfa
            # scaled by game weight
            diff = (y - p_home) * w
            grad_r[h_idx] += diff
            grad_r[a_idx] -= diff
            if not is_neutral:
                grad_hfa += diff

        # update
        for i in range(n):
            ratings[i] += lr * grad_r[i]
            # clamp to avoid runaway values
            if ratings[i] > max_abs:
                ratings[i] = max_abs
            elif ratings[i] < -max_abs:
                ratings[i] = -max_abs

        hfa += lr * grad_hfa
        if hfa > max_abs:
            hfa = max_abs
        elif hfa < -max_abs:
            hfa = -max_abs

    # map back to team_id -> rating
    id_to_rating = {tid: ratings[idx] for tid, idx in team_index.items()}
    return id_to_rating, hfa


# --------------------------------------------------------------------
# Win–loss / final rating construction
# --------------------------------------------------------------------

def compute_record(
    teams: List[Dict[str, Any]],
    games: List[Dict[str, Any]]
) -> Dict[int, Dict[str, Any]]:
    """
    Compute wins, losses, games, win_pct per team_id.
    """
    rec: Dict[int, Dict[str, Any]] = {}
    for t in teams:
        tid = t["team_id"]
        rec[tid] = {"wins": 0, "losses": 0, "games": 0}

    for g in games:
        h = g["home_team_id"]
        a = g["away_team_id"]
        hs = g["home_score"]
        as_ = g["away_score"]

        rec[h]["games"] += 1
        rec[a]["games"] += 1

        if hs > as_:
            rec[h]["wins"] += 1
            rec[a]["losses"] += 1
        elif as_ > hs:
            rec[a]["wins"] += 1
            rec[h]["losses"] += 1
        else:
            # tie: count as 0.5 win / 0.5 loss, or ignore; here we ignore for simplicity
            pass

    for tid, r in rec.items():
        g = r["games"]
        if g > 0:
            r["win_pct"] = r["wins"] / g
        else:
            r["win_pct"] = 0.0

    return rec


def build_final_ratings(
    teams: List[Dict[str, Any]],
    base_ratings: Dict[int, float],
    records: Dict[int, Dict[str, Any]],
    cfg: Dict[str, Any],
) -> Dict[int, float]:
    """
    Combine base rating with win-loss-based adjustments.

    final_rating = base_rating
                 + wl_weight * (win_pct - avg_win_pct)
                 + undefeated_bonus_if_applicable
    """
    win_pcts = [r["win_pct"] for r in records.values() if r["games"] > 0]
    avg_win_pct = sum(win_pcts) / len(win_pcts) if win_pcts else 0.5

    wl_weight = cfg["wl_weight"]
    undefeated_min_games = cfg["undefeated_min_games"]
    undefeated_bonus = cfg["undefeated_bonus"]

    final: Dict[int, float] = {}

    for t in teams:
        tid = t["team_id"]
        base = base_ratings.get(tid, 0.0)
        rec = records[tid]
        win_pct = rec["win_pct"]

        wl_term = wl_weight * (win_pct - avg_win_pct)

        # Optional undefeated bump
        if rec["games"] >= undefeated_min_games and rec["losses"] == 0 and rec["wins"] > 0:
            extra = undefeated_bonus
        else:
            extra = 0.0

        final_rating = base + wl_term + extra
        final[tid] = final_rating

    return final


# --------------------------------------------------------------------
# Head-to-head ordering refinement
# --------------------------------------------------------------------

def build_game_lookup(games: List[Dict[str, Any]]) -> Dict[Tuple[int, int], Dict[str, Any]]:
    """
    Map (winner_id, loser_id) -> a representative game dict (latest one).
    If teams played multiple times, we keep the last game chronologically.
    """
    lookup: Dict[Tuple[int, int], Dict[str, Any]] = {}
    for g in games:
        h = g["home_team_id"]
        a = g["away_team_id"]
        hs = g["home_score"]
        as_ = g["away_score"]
        if hs == as_:
            continue

        if hs > as_:
            w, l = h, a
        else:
            w, l = a, h

        key = (w, l)
        existing = lookup.get(key)
        if existing is None or parse_date(g["date"]) > parse_date(existing["date"]):
            lookup[key] = g
    return lookup


def apply_head_to_head(
    teams_ranked: List[Dict[str, Any]],
    games: List[Dict[str, Any]],
    ratings: Dict[int, float],
    cfg: Dict[str, Any],
) -> List[Dict[str, Any]]:
    """
    Given an initial ranking (sorted list of team dicts), apply a simple
    head-to-head "swap if close" rule, using the given ratings.

    If team B beat team A and rating(B) <= rating(A) + margin_threshold,
    then move B above A.
    """
    swap_margin = cfg["h2h_swap_margin"]
    game_lookup = build_game_lookup(games)

    # We'll do a simple bubble-pass over the list; for 136 teams this is cheap.
    changed = True
    while changed:
        changed = False
        for i in range(len(teams_ranked) - 1):
            t_a = teams_ranked[i]
            t_b = teams_ranked[i + 1]
            id_a = t_a["team_id"]
            id_b = t_b["team_id"]

            key_ab = (id_a, id_b)
            key_ba = (id_b, id_a)

            r_a = ratings[id_a]
            r_b = ratings[id_b]

            # Did B beat A?
            g_ba = game_lookup.get(key_ba)
            if g_ba is not None:
                # If B's rating is not sufficiently above A (or even below),
                # we allow B to jump A to respect head-to-head.
                if r_b + swap_margin >= r_a:
                    teams_ranked[i], teams_ranked[i + 1] = teams_ranked[i + 1], teams_ranked[i]
                    changed = True

            # You could add symmetrical logic for A beating B, but the base
            # sort already tends to favor the stronger team; we only need to
            # "fix" cases where the direct winner is oddly below.

    return teams_ranked


# --------------------------------------------------------------------
# Ranking assembly
# --------------------------------------------------------------------

def assemble_rankings(
    teams: List[Dict[str, Any]],
    games: List[Dict[str, Any]],
    cfg: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Main pipeline:
      1) fit base ratings
      2) compute W-L records
      3) build final ratings (base + win-loss adjustments)
      4) make a raw ranking sorted by final rating
      5) apply head-to-head refinement
    """
    base_ratings, hfa = fit_base_ratings(teams, games, cfg)
    records = compute_record(teams, games)
    final_ratings = build_final_ratings(teams, base_ratings, records, cfg)

    latest_date_dt = compute_latest_game_date(games)
    latest_date_str = latest_date_dt.date().isoformat() if latest_date_dt else None

    # build full team data objects
    enriched = []
    for t in teams:
        tid = t["team_id"]
        rec = records[tid]
        enriched.append({
            "team_id": tid,
            "team_name": t["team_name"],
            "mascot": t.get("mascot", ""),
            "conference": t.get("conference", ""),
            "wins": rec["wins"],
            "losses": rec["losses"],
            "games": rec["games"],
            "win_pct": rec["win_pct"],
            "base_rating": base_ratings.get(tid, 0.0),
            "rating": final_ratings[tid],  # final rating used for ranking
        })

    # raw ranking: by final rating only
    raw_sorted = sorted(enriched, key=lambda x: x["rating"], reverse=True)

    # h2h ranking: apply head-to-head adjustments on top of raw_sorted
    h2h_sorted = apply_head_to_head(raw_sorted.copy(), games, final_ratings, cfg)

    return {
        "home_field_adv": hfa,
        "latest_game_date": latest_date_str,
        "raw_ranking": raw_sorted,
        "h2h_ranking": h2h_sorted,
    }


# --------------------------------------------------------------------
# AWS Lambda handler
# --------------------------------------------------------------------

SAMPLE_TEAMS = load_from_s3("static-assets/teams.json")
SAMPLE_GAMES = load_from_s3("static-assets/games.json")

def lambda_handler(event, context):
    """
    Expected event format:

    {
        "teams": [
            {"team_id": 1, "team_name": "BYU", "mascot": "Cougars", "conference": "Big 12"},
            ...
        ],
        "games": [
            {
                "game_id": 1,
                "date": "2025-08-31",
                "home_team_id": 1,
                "away_team_id": 2,
                "home_score": 28,
                "away_score": 24,
                "neutral_site": 0
            },
            ...
        ],
        "config_overrides": {
            ... optional subset of CONFIG keys ...
        }
    }

    Returns a JSON-serializable dict with:
      - home_field_adv
      - latest_game_date
      - raw_ranking
      - h2h_ranking
    """
    teams = event.get("teams", [])
    games = event.get("games", [])
    if not teams or not games:
        return {
            "error": "Missing 'teams' or 'games' in event.",
            "received_keys": list(event.keys()),
        }

    # allow runtime overrides of CONFIG
    cfg = CONFIG.copy()
    overrides = event.get("config_overrides", {})
    for k, v in overrides.items():
        if k in cfg:
            cfg[k] = v

    result = assemble_rankings(teams, games, cfg)
    final_sorted = sorted(result["h2h_ranking"], key=lambda t: t["rating"], reverse=True)

    return result


# --------------------------------------------------------------------
# Local testing helper
# --------------------------------------------------------------------

if __name__ == "__main__":

    event = {
        "teams": SAMPLE_TEAMS,
        "games": SAMPLE_GAMES,
        # optional overrides:
        # "config_overrides": {"wl_weight": 3.0, "undefeated_bonus": 1.0},
    }

    out = lambda_handler(event, None)
    print(json.dumps(out, indent=2))


{
  "home_field_adv": 0.4842266602303075,
  "latest_game_date": "2025-11-23",
  "raw_ranking": [
    {
      "team_id": 81,
      "team_name": "Ohio State",
      "mascot": "Buckeyes",
      "conference": "Big Ten",
      "wins": 10,
      "losses": 0,
      "games": 10,
      "win_pct": 1.0,
      "base_rating": 1.736006647291985,
      "rating": 3.4916405052514246
    },
    {
      "team_id": 42,
      "team_name": "Indiana",
      "mascot": "Hoosiers",
      "conference": "Big Ten",
      "wins": 10,
      "losses": 0,
      "games": 10,
      "win_pct": 1.0,
      "base_rating": 1.6617201201947371,
      "rating": 3.4173539781541766
    },
    {
      "team_id": 107,
      "team_name": "Texas A&M",
      "mascot": "Aggies",
      "conference": "SEC",
      "wins": 10,
      "losses": 0,
      "games": 10,
      "win_pct": 1.0,
      "base_rating": 1.4595664615328996,
      "rating": 3.215200319492339
    },
    {
      "team_id": 79,
      "team_name": "Notre Dame",
      "mascot"